In [2]:
import requests
import pandas as pd

# Petfinder API credentials
API_KEY = "zHREaHuUnVsqokhiKtJEjoyFem5jjmP6ehAmiXaf7H8M2iqvYg"
API_SECRET = "JOg0KOJ5hTq7pEHTFAKlQHYIoElAL9H6jOJuNrGb"

def get_auth_token():
    """Authenticate with the Petfinder API and return the access token."""
    url = "https://api.petfinder.com/v2/oauth2/token"
    data = {
        "grant_type": "client_credentials",
        "client_id": API_KEY,
        "client_secret": API_SECRET,
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    return response.json()["access_token"]

def fetch_adoptable_pets(cities, token):
    """Fetch pets information for specified cities, filtered by a specific status post-fetch."""
    url = "https://api.petfinder.com/v2/animals"
    headers = {"Authorization": f"Bearer {token}"}
    pets_data = []

    for city in cities:
        for animal_type in ['Cat', 'Dog']:
            params = {
                "type": animal_type,
                "location": city,
                "limit": 100,
            }
            page = 1

            while True:
                params["page"] = page
                response = requests.get(url, headers=headers, params=params)
                response.raise_for_status()
                data = response.json()

                for animal in data["animals"]:
                    if animal["status"] == "adopted":
                        pets_data.append({
                            "type": animal["type"],
                            "breed": animal["breeds"]["primary"],
                            "location": animal["contact"]["address"]["city"],
                            "color": animal["colors"]["primary"] if animal["colors"]["primary"] else "N/A",
                            "size": animal["size"],
                            "date_posted": animal["published_at"],
                            "name": animal["name"],
                            "status": animal["status"],
                            "gender": animal["gender"]
                        })

                if page >= data["pagination"]["total_pages"]:
                    break
                page += 1

    return pd.DataFrame(pets_data)

# Cities to get data for
cities = ["New York City, NY", "Los Angeles, CA", "Chicago, IL", "Houston, TX", "Phoenix, AZ"]

token = get_auth_token()

# Fetch pets data
pets_df = fetch_adoptable_pets(cities, token)

# Display the DataFrame
print(pets_df.head())

Empty DataFrame
Columns: []
Index: []
